### Install pre-requisite packages for the course

Please ensure that this cell is run, to install all pre-requisite packages needed for subsequent code examples.

In [14]:
!pip install python-dotenv
!pip install llama-index
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai
!pip install llama-index-llms-azure-openai
!pip install llama-index-embeddings-azure-openai
!pip install llama-index-agent-introspective
!pip install llama-index-program-openai
!pip install llama-index-readers-file
!pip install pydantic
!pip install llama-index-core
!pip install llama-index-utils-workflow
!pip install llama-index-tools-wikipedia
!pip install llama-index-readers-json
!pip install llama-index-readers-file
!pip install pymupdf
!pip install llama-index-tools-google

  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 8.5 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/12.6 MB 3.7 MB/s eta 0:00:04
   ------ --------------------------------- 2.1/12.6 MB 4.1 MB/s eta 0:00:03
   ---------- ----------------------------- 3.1/12.6 MB 4.3 MB/s eta 0:00:03
   -------------- ------------------------- 4.5/12.6 MB 4.6 MB/s eta 0:00:02
   ---------------- ----------------------- 5.2/12.6 MB 4.7 MB/s eta 0:00:02
   -------------------- ------------------- 6.3/12.6 MB 4.6 MB/s et

### 01.04. Setup LLM and Tools in LlamaIndex

#### Setup the LLM

In [2]:
#Using AzureOpenAI as LLM. Other LLM options are available in LlamaIndex
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()

#API key for AzureOpenAI. Replace with your own key
api_key = "4e4ab31800a64ae892cbb768fe28c0fc"
#Azure Endpoint URL. Replace with your own URL
azure_endpoint = "https://agentic-ai-course-kumaran.openai.azure.com/"

#Note : Function calling support only available in GPT-4+
#Create the LLM object
llm=AzureOpenAI(
    model="gpt-4o",
    deployment_name="gpt-4o",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version="2024-08-01-preview",
)

#### Set up Functions

In [3]:
#define a couple of functions
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y

### 01.05. Setup and execute an AI Agent

In [7]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.base import ReActAgent

#Create function tools 
sum_tool = FunctionTool.from_defaults(find_sum)
product_tool=FunctionTool.from_defaults(find_product)

#Create a pre-built ReAct agent
simple_agent = ReActAgent.from_tools(
                [sum_tool, product_tool],
                llm=llm,
                verbose=True #Set verbose for detailed logs
                )

#### Test the Agent

In [8]:
response = simple_agent.chat("What is 2 + 2? ")

> Running step 2c7e3a73-f2bb-47dd-a1d5-1dbffc8512b4. Step input: What is 2 + 2? 
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_sum
Action Input: {'x': 2, 'y': 2}
Observation: 4
> Running step 6ee0190b-e067-4fb1-8af1-9cbc5a2619ed. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 2 + 2 is 4.


In [9]:
response = simple_agent.chat("Multiple 2 by 3 and then add 4")

> Running step b81f2fc3-c663-40fa-a842-15d0cbaa8329. Step input: Multiple 2 by 3 and then add 4
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_product
Action Input: {'x': 2, 'y': 3}
Observation: 6
> Running step 77bdd929-2e8e-49e7-8c34-6664d816daf4. Step input: None
Thought: I have the product of 2 and 3, which is 6. Now, I need to add 4 to this result.
Action: find_sum
Action Input: {'x': 6, 'y': 4}
Observation: 10
> Running step 5056c537-96e7-490c-89d6-47f941d70b26. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of multiplying 2 by 3 and then adding 4 is 10.
